In [0]:
# Gold Layer: Create KPI Views  
# Builds business-ready views on top of the Silver Delta table.

from pyspark.sql import SparkSession

In [0]:
# Path config
SILVER_PATH = "/mnt/kardia/silver/silver_patients"

In [0]:
# Initialize a Spark session with minimal shuffle partitions
spark = (
    SparkSession.builder
      .appName("your_notebook_name")
      .config("spark.sql.shuffle.partitions", "1")
      .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
      .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
      .getOrCreate()
)
spark.sparkContext.setLogLevel("ERROR")

In [0]:
# 1️. Read Silver and register a temp view
df_silver = spark.read.format("delta").load(SILVER_PATH)
df_silver.createOrReplaceTempView("tmp_silver_patients")

# 2️. Build gender-breakdown KPI view
spark.sql("""
CREATE OR REPLACE TEMP VIEW vw_gender_breakdown AS
SELECT
  GENDER,
  COUNT(*) AS cnt
FROM tmp_silver_patients
GROUP BY GENDER
""")


In [0]:
# Quick sanity check
print("vw_gender_breakdown preview:")
display(spark.table("vw_gender_breakdown"))